In [ ]:
output_path = "./inference_ouput"

import os

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
# 遍历打开 mcts_output 目录下的所有 json 文件
import json

def process_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    return data

iteration_consider = 40

for filename in os.listdir('./mcts_output'):
    if filename.endswith('.json'):
        result_dict = {}

        file_path = os.path.join('./mcts_output', filename)
        data = process_json_file(file_path)

        # 选择 data['all_candidates']['final_answer'] 中出现频率最高的作为答案
        from collections import Counter
        
        final_answers = []
        for candidate in data['all_candidates']:
            if 'found_in_round' in candidate:
                if candidate['found_in_round'] < iteration_consider:
                    final_answers.append(candidate['final_answer'])
            else:
                final_answers.append(candidate['final_answer'])

        answer_counter = Counter(final_answers)
        if not answer_counter:
            print(f"No valid answers found in {filename}. Skipping.")
            continue
        most_common_answer, _ = answer_counter.most_common(1)[0]
        print(f"Most common answer in {filename}: {most_common_answer}")

        result_dict['formatted_answer'] = most_common_answer
        #result_dict['task'] = data['task']

        # 将 result_dict 写入 output_path 目录下的 json 文件，文件名为原文件名替换 _candidates.json 为 .json
        output_filename = filename.replace('_candidates.json', '.json')
        output_file_path = os.path.join(output_path, output_filename)
        with open(output_file_path, 'w') as output_file:
            json.dump(result_dict, output_file, indent=4)